 # Схема базы данных

![Схема базы данных](https://pictures.s3.yandex.net/resources/13_border2880_1628581406.png)

1.
Посчитайте, сколько компаний закрылось.

In [ ]:
SELECT COUNT(DISTINCT id)
FROM company
WHERE status = 'closed';

2.
Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы `company`. Отсортируйте таблицу по убыванию значений в поле `funding_total` .

In [ ]:
SELECT SUM(funding_total)
FROM company
WHERE category_code = 'news'
  AND country_code = 'USA'
GROUP BY name
ORDER BY SUM(funding_total) DESC;

3.
Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash'
  AND CAST(acquired_at AS date) BETWEEN '2011-01-01' AND '2013-12-31';

4.
Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на `'Silver'`.

In [ ]:
SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%';

5.
Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку `'money'`, а фамилия начинается на `'K'`.

In [ ]:
SELECT *
FROM people
WHERE twitter_username LIKE '%money%'
  AND last_name LIKE 'K%';

6.
Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:
SELECT country_code,
       SUM(funding_total)
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC;

7.
Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
SELECT CAST(funded_at AS date),
       MIN(raised_amount),
       MAX(raised_amount)
FROM funding_round
GROUP BY CAST(funded_at AS date)
HAVING MIN(raised_amount)!= 0
   AND MIN(raised_amount)!= MAX(raised_amount);

8.
Создайте поле с категориями:
Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию `high_activity`.
Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию `middle_activity`.
Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию `low_activity`.
Отобразите все поля таблицы `fund`и новое поле с категориями.

In [ ]:
SELECT *,
       CASE
           WHEN invested_companies >= 100 THEN 'high_activity'
           WHEN invested_companies >= 20 AND invested_companies < 100  THEN 'middle_activity'
           ELSE 'low_activity'
        END
FROM fund;

9.
Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
SELECT 
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
       ROUND(AVG(investment_rounds))
FROM fund
GROUP BY activity
ORDER BY ROUND(AVG(investment_rounds));


10.
Выгрузите таблицу с десятью самыми активными инвестирующими странами. Активность страны определите по среднему количеству компаний, в которые инвестируют фонды этой страны.
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды, основанные с 2010 по 2012 год включительно.
Исключите из таблицы страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. Отсортируйте таблицу по среднему количеству компаний от большего к меньшему, а затем по коду страны в лексикографическом порядке.

In [ ]:
SELECT country_code,
       MIN(invested_companies),
       MAX(invested_companies),
       AVG(invested_companies)
FROM fund
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date))  BETWEEN '2010' AND '2012'
GROUP BY country_code
HAVING MIN(invested_companies)!= 0
ORDER BY AVG(invested_companies) DESC
LIMIT 10;

11.
Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
SELECT p.first_name,
       p.last_name,
       e.instituition
FROM people AS P
LEFT JOIN education AS e ON p.id = e.person_id;

12.
Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [ ]:
SELECT c.name,
       COUNT(DISTINCT e.instituition)
FROM company AS c
INNER JOIN people AS p ON c.id = p.company_id
INNER JOIN education AS e ON p.id = e.person_id
GROUP BY c.name
ORDER BY COUNT(DISTINCT e.instituition) DESC
LIMIT 5;

13.
Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
SELECT DISTINCT name
FROM company 
WHERE id IN (SELECT company_id
             FROM funding_round
             WHERE is_first_round = 1
               AND is_last_round = 1)
  AND status = 'closed';

14.
Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
SELECT DISTINCT id
FROM people
WHERE company_id IN  (SELECT DISTINCT id
                      FROM company 
                      WHERE id IN (SELECT company_id
                                   FROM funding_round
                                   WHERE is_first_round = 1
                                     AND is_last_round = 1)
                      AND status = 'closed'); 

15.
Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
SELECT DISTINCT person_id,
       instituition
FROM education
WHERE person_id IN (SELECT DISTINCT id
                    FROM people
                    WHERE company_id IN  (SELECT DISTINCT id
                                          FROM company 
                                          WHERE id IN (SELECT company_id
                                          FROM funding_round
                                          WHERE is_first_round = 1
                                            AND is_last_round = 1)
                      AND status = 'closed'));

16.
Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [ ]:
SELECT DISTINCT person_id,
       COUNT(instituition)
FROM education
WHERE person_id IN (SELECT DISTINCT id
                    FROM people
                    WHERE company_id IN  (SELECT DISTINCT id
                                          FROM company 
                                          WHERE id IN (SELECT company_id
                                          FROM funding_round
                                          WHERE is_first_round = 1
                                            AND is_last_round = 1)
                      AND status = 'closed'))
GROUP BY person_id;

17.
Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
SELECT AVG(a.instituit)
FROM (SELECT DISTINCT person_id,
           COUNT(instituition) AS instituit
    FROM education
    WHERE person_id IN (SELECT DISTINCT id
                    FROM people
                    WHERE company_id IN  (SELECT DISTINCT id
                                          FROM company 
                                          WHERE id IN (SELECT company_id
                                          FROM funding_round
                                          WHERE is_first_round = 1
                                            AND is_last_round = 1)
                      AND status = 'closed'))
GROUP BY person_id) AS a;

18.
Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook*.
*(сервис, запрещённый на территории РФ)

In [ ]:
SELECT AVG(a.instituit)
FROM
(SELECT person_id,
       COUNT(instituition) AS instituit
FROM education
WHERE person_id IN (SELECT id
                    FROM people
                    WHERE company_id in (SELECT id
                                         FROM company
                                         WHERE name LIKE 'Facebook'))
GROUP BY person_id) AS a;     

19.
Составьте таблицу из полей:
 - `name_of_fund` — название фонда;
 - `name_of_company` — название компании;
 - `amount` — сумма инвестиций, которую привлекла компания в раунде.
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
WITH
a AS ( SELECT id,
       company_id,
       raised_amount
FROM funding_round
WHERE company_id IN (SELECT id
                     FROM company
                     WHERE milestones > 6)
  AND EXTRACT(YEAR FROM CAST(funded_at AS date)) BETWEEN '2012' AND '2013')  
SELECT f.name AS name_of_fund,
       c.name AS name_of_company,
       a.raised_amount AS amount
FROM a
JOIN company AS c ON a.company_id = c.id 
JOIN investment AS i ON a.id = i.funding_round_id
JOIN fund AS f ON i.fund_id = f.id;

20.
Выгрузите таблицу, в которой будут такие поля:
 - название компании-покупателя;
 - сумма сделки;
 - название компании, которую купили;
 - сумма инвестиций, вложенных в купленную компанию;
 - доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.

Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы.

Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. 

Ограничьте таблицу первыми десятью записями.

In [ ]:
SELECT c.name AS acquiring_company,
       ac.price_amount AS price,
       co.name AS acquired_company,
       co.funding_total AS investment,
       ROUND(ac.price_amount/co.funding_total) AS percent
FROM acquisition AS ac
LEFT OUTER JOIN company AS c ON ac.acquiring_company_id = c.id 
LEFT OUTER JOIN company AS co ON ac.acquired_company_id = co.id
WHERE ac.price_amount!=0
  AND co.funding_total!=0
ORDER BY price DESC, acquired_company
LIMIT 10;

21.
Выгрузите таблицу, в которую войдут названия компаний из категории `social`, получившие финансирование с 2010 по 2013 год включительно. Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
SELECT c.name,
       EXTRACT(MONTH FROM CAST(fr.funded_at AS date))
FROM company AS c
RIGHT OUTER JOIN funding_round AS fr ON c.id = fr.company_id
WHERE c.category_code = 'social'
  AND EXTRACT(YEAR FROM CAST(fr.funded_at AS date)) BETWEEN '2010' AND '2013';

22.
Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
 - номер месяца, в котором проходили раунды;
 - количество уникальных названий фондов из США, которые инвестировали в этом месяце;
 - количество компаний, купленных за этот месяц;
 - общая сумма сделок по покупкам в этом месяце.

In [ ]:
WITH
a AS (SELECT EXTRACT(MONTH FROM CAST(fr.funded_at AS date)) AS month, 
       COUNT(DISTINCT f.name) AS fond_usa
FROM investment AS i 
LEFT OUTER JOIN funding_round AS fr ON i.funding_round_id = fr.id
LEFT OUTER JOIN fund AS f ON i.fund_id = f.id
WHERE EXTRACT(YEAR FROM CAST(fr.funded_at AS date)) BETWEEN '2010' AND '2013'
  AND f.country_code = 'USA'
GROUP BY EXTRACT(MONTH FROM CAST(fr.funded_at AS date))),
b AS (SELECT EXTRACT(MONTH FROM CAST(acquired_at AS date)) AS month, 
       COUNT(acquired_company_id) AS acquired_company,
       SUM(price_amount) AS total_sum
FROM acquisition 
WHERE EXTRACT(YEAR FROM CAST(acquired_at AS date)) BETWEEN '2010' AND '2013'
GROUP BY EXTRACT(MONTH FROM CAST(acquired_at AS date)))
SELECT a.month,
       a.fond_usa,
       b.acquired_company,
       b.total_sum
FROM a JOIN b ON a.month = b.month;

23.
Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
WITH
a AS (SELECT country_code AS country,
             AVG(funding_total) AS year_2011
      FROM company
      WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2011
      GROUP BY country_code),
b AS (SELECT country_code AS country,
             AVG(funding_total) AS year_2012
      FROM company
      WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2012
      GROUP BY country_code),
c AS (SELECT country_code AS country,
             AVG(funding_total) AS year_2013
      FROM company
      WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2013
      GROUP BY country_code)
SELECT a.country,
       a.year_2011,
       b.year_2012,
       c.year_2013
FROM a JOIN b ON a.country = b.country
JOIN c ON a.country = c.country
ORDER BY a.year_2011 DESC;